## Vanilla Recurrent Neural Network implementation

In [ ]:
#import libraries
import os
os.environ["THEANO_FLAGS"] = "device=gpu,floatX=float32,exception_verbosity='high'"

from theano import *
import theano.tensor as T

import numpy as np
import pandas as pd

### Loading data

In [ ]:
tsx = pd.read_csv('SPY.csv')

#difference and normalize data
X = tsx.values[:,1:-2]
X = tsx[['Open','High','Low','Close','Adj Close','Volume']]
Xd = np.diff(X,axis=0)
Xsd = ((Xd - np.mean(Xd,axis=0)) / np.std(Xd.astype(np.float32),axis=0)).astype(np.float32)[:-1][-5365:]

#up/down label
Y = ((Xd[:,4] > 0)*1)[1:].astype(np.int32)[-5365:].reshape(-1,1)

### Building vanilla RNN

In [ ]:
#paramters
n_in = 6
rl = [36]
seed = 132345
activation = T.nnet.relu

####
#computational flow
rng = np.random.RandomState(seed)
x = T.matrix('x')
y = T.imatrix('y')
def genW(size):
    return np.asarray(
        rng.uniform(
            low=-0.1,
            high=0.1,
            size = size
        ),
        dtype = theano.config.floatX)
#recurrent block
M = n_in
N = rl[-1]
nr = len(rl)
W0 = theano.shared(value=genW((M,rl[0])),name="W0",borrow=True)
Wu = theano.shared(value=genW((rl[-1],rl[0])),name="Wu",borrow=True)
b0 = theano.shared(value=np.zeros(rl[0],dtype=theano.config.floatX),name="b0",borrow=True)
u0 = theano.shared(value=np.zeros((1,N),dtype=theano.config.floatX),borrow=True)
params = [W0,Wu,b0]
W = []
b = []
for i in range(0,nr-1):
    W.append(
        theano.shared(value=genW((rl[i],rl[i+1])),name="W"+str(i+1),borrow=True))
    b.append(
        theano.shared(value=np.zeros(rl[i+1],dtype=theano.config.floatX),name="b"+str(i+1),borrow=True))
    params += [W[i], b[i]]
#recurrent step
def recurrence(xt,u_t1):
    u_t = T.tanh(T.dot(xt,W0) + T.dot(u_t1,Wu) + b0)
    for i in range(nr-1):
        u_t = activation(T.dot(u_t,W[i]) + b[i])
    return u_t,OrderedUpdates({u0: u_t})
#scan for recurrent computation
u_t, update_u0 = theano.scan(fn=recurrence,
                              sequences = [x],
                              outputs_info = [u0]
                             )
#recurrent output
rnn_out = u_t[:,0,:]
#output layer
Wout = theano.shared(value=np.zeros((N,1),dtype=np.float32),name="Wout",borrow=True)
bout = theano.shared(value=np.zeros(1,dtype=np.float32),name="bout",borrow=True)
params += [Wout,bout]
#output probability
p_y_x = T.nnet.sigmoid(T.dot(rnn_out,Wout) + bout)
#label prediction
y_p = T.gt(p_y_x,0.5)*1
#cross entropy loss
CE = T.nnet.binary_crossentropy(p_y_x,y).mean()
#reset hidden states
def reset_hiddens():
    u0.set_value(np.zeros((1,N),dtype=theano.config.floatX))

### Training

In [ ]:
#paramters
tlength = 5000
learning_rate=0.5
n_epochs=5000
sX = theano.shared(np.asarray(Xsd[:tlength], dtype=theano.config.floatX))
sY = theano.shared(np.asarray(Y[:tlength], dtype=np.int32))

####
#function for training
cost = CE
gparams = [T.grad(cost,param) for param in params]
updates = [(param, param - learning_rate * gparam) for param,gparam in zip(params,gparams)]
train_rnn = theano.function(
    inputs=[],
    outputs=[cost],
    updates=updates,
    givens={
        x: sX,
        y: sY
    },
)
#function for predicting
pred_y = theano.function(
    inputs=[x],
    outputs=y_p
)

In [ ]:
#training
print "training..."

for ep in range(n_epochs):
        print('Training epoch %d, cost %f' % (ep, np.mean(train_rnn())))

In [ ]:
#testing
print np.mean(pred_y(Xsd)[tlength:]==Y[tlength:])